In [2]:
import numpy as np
import matplotlib.pyplot as plt
import lmfit.models

## Exercise 1

In [3]:
np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)

## Exercise 2

In [4]:
np.random.seed(2)
x = np.linspace(-5 , 5, 50)
yerr = 0.1+np.abs(0.1*np.random.normal(size = x.shape))

y = 5/(np.sqrt(2*np.pi*1.0))*np.exp(-0.5*x**2.0/1.0)+yerr*np.random.normal(size = x.shape)

## Exercise 3

In [5]:
def weird_model(x, a, b, c):
    return a*np.sin(10*x) + b*np.exp(x)+ c*np.log(x)

In [6]:
np.random.seed(2)
x = np.linspace(0.1 , 2, 100)
yerr = 0.5+np.abs(0.5*np.random.normal(size = x.shape))
y = weird_model(x, 2, 0.5, 3)+yerr*np.random.normal(size = x.shape)

## Exercise 4

## Exercise 5

In [7]:
from functions import gwfreq, osc_scale, osc,lower, higher

In [8]:
from gwpy.timeseries import TimeSeries
import h5py
from scipy.interpolate import interp1d

import os

fn = 'H-H1_GWOSC_16KHZ_R1-1126257415-4096.hdf5' # data file
tevent = 1126259462.422 # Mon Sep 14 09:50:45 GMT 2015
evtname = 'GW150914' # event name

detector = 'H1' # detecotr: L1 or H1

strain = TimeSeries.read(fn, format='hdf5.losc')
center = int(tevent)
strain = strain.crop(center-16, center+16)
asd = strain.asd(fftlength = 1)

NRtime, NR_H1 = np.genfromtxt('GW150914_4_NR_waveform.txt').transpose()
nrdata= TimeSeries(NR_H1, times = NRtime)

In [9]:
def whiten_and_bandpass(strain, asd, bp_low, bp_high):
    fft = strain.fft()
    asdinterp = interp1d(asd.frequencies, asd)
    asddiv =asdinterp(fft.frequencies)
    white_freq = fft/asddiv
    white = white_freq.ifft()
    whitebp = white.bandpass(bp_low,bp_high)
    return TimeSeries(whitebp, t0=strain.t0)